In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from datetime import datetime, timedelta
import random

spark = SparkSession.builder.getOrCreate()

# -----------------------------
# 1. Load Enriched Bronze Data
# -----------------------------
df_inventory = spark.read.format("delta").load("/mnt/delta/bronze/inventory")
df_shipments = spark.read.format("delta").load("/mnt/delta/bronze/shipments")
df_vendors = spark.read.format("delta").load("/mnt/delta/bronze/vendors")

# ----------------------------------
# 2. Add shipment_date if needed
# ----------------------------------
@udf(DateType())
def random_date():
    base = datetime(2025, 6, 1)
    return base + timedelta(days=random.randint(0, 14))

if "shipment_date" not in df_shipments.columns:
    df_shipments = df_shipments.withColumn("shipment_date", random_date())

# -----------------------
# 3. Write Enriched Tables
# -----------------------
df_inventory.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/silver/inventory_enriched")
df_shipments.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/silver/shipments_enriched")
df_vendors.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/mnt/delta/silver/vendors_clean")

# -----------------------
# 3.5 Write Enriched Inventory
# ----------------------
from pyspark.sql.functions import when

df_inventory = spark.read.format("delta").load("/mnt/delta/silver/inventory_enriched")

df_inventory = df_inventory.withColumn(
    "vendor_id",
    when(col("item_id") == "ITM001", "V001")
    .when(col("item_id") == "ITM002", "V002")
    .when(col("item_id") == "ITM003", "V003")
    .when(col("item_id") == "ITM004", "V004")
    .when(col("item_id") == "ITM005", "V005")
    .when(col("item_id") == "ITM006", "V006")
    .when(col("item_id") == "ITM007", "V007")
    .when(col("item_id") == "ITM008", "V008")
    .when(col("item_id") == "ITM009", "V009")
    .otherwise("V010")
)


# ---------------------------------------------------------
# 4. Join Inventory + Shipments + Vendors on vendor_id
# ---------------------------------------------------------
df_joined = (
    df_inventory.alias("inv")
    .join(df_shipments.alias("ship"), on="vendor_id", how="inner")
    .join(df_vendors.alias("vend"), on="vendor_id", how="left")
    .select(
        "inv.vendor_id",
        "inv.item_id", "inv.item_name", "inv.quantity_on_hand", "inv.reorder_level", "inv.last_updated",
        "ship.shipment_id", "ship.ship_date", "ship.shipment_date", "ship.destination", "ship.status",
        col("vend.name").alias("vendor_name"),
        "vend.location", "vend.rating"
    )
)


df_joined.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .save("/mnt/delta/silver/inventory_shipments_joined_clean")
